In [2]:
import numpy as np
import random
import math
import multiprocess as mp
import talib as ta
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime,date
from gga import GGA
from island_ga import IslandGGA  
from chromosome import Chromosome 
import itertools

In [15]:
data=pd.read_csv('UA.csv')
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [23]:
#rename column to remove spaces in column names
data=pd.read_csv('UA.csv')
data =data.rename(columns={'Close': 'close', 'Volume': 'volume',
                           'Open':'open','High':'high','Low':'low'})
#change date string into date format and sort the dataframe in ascending order
data[['Date']] = data[['Date']].applymap(lambda x: datetime.strptime(x,'%Y-%m-%d').date())
data =data.sort_values(by='Date')
#Create Signals using Talib library
data['5EMA'] = ta.SMA(np.array(data['close']),5)
data['20EMA'] = ta.EMA(np.array(data['close']), timeperiod = 20)
data['RSI'] = ta.RSI(np.array(data['close']), timeperiod = 14)
data['WILLR'] = ta.WILLR(np.array(data['high']), np.array(data['low']), np.array(data['close']), timeperiod=14)
data['MOM'] = ta.MOM(np.array(data['close']), timeperiod=5)
data['CCI'] = ta.CCI(np.array(data['high']), np.array(data['low']), np.array(data['close']), timeperiod=14)
data['SLOWK'],data['SLOWD'] = ta.STOCH(np.array(data['high']), np.array(data['low']), np.array(data['close']), fastk_period=14, slowk_period=3, slowd_period=3)
data['MACD'],data['MACDSIGNAL'],data['MACDHIST'] = ta.MACD(np.array(data['close']), fastperiod=12, slowperiod=26, signalperiod=9)
data['DMI'] = ta.DX(np.array(data['high']), np.array(data['low']), np.array(data['close']), timeperiod=14)   

In [9]:
data.head()

,Date,open,high,low,close,Adj Close,volume,5EMA,20EMA,RSI,WILLR,MOM,CCI,SLOWK,SLOWD,MACD,MACDSIGNAL,MACDHIST,DMI
0,2016-04-01,40.800400,40.800400,40.800400,40.800400,40.800400,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-04-04,40.482655,41.862858,40.462795,40.462795,40.462795,956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-04-05,40.214558,41.733772,40.214558,40.800400,40.800400,1712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-04-06,41.058567,41.703983,40.770611,41.505394,41.505394,721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-04-07,41.793350,41.833069,41.267086,41.267086,41.267086,5958,40.967215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#  """Based on parameter setting adopted in Chen et al (2021) """

def generate_candidate_trading_signals(data):
    """Based on parameter setting adopted in Chen et al (2021) """
    conditions ={'TS1_CONDITIONS':[
                (data['5EMA'] > data['20EMA']),
                (data['5EMA'] < data['20EMA'])],
                 'TS2_CONDITIONS':[
                (data['RSI'] > 30),
                (data['RSI'] < 70),
                ],
                 'TS3_CONDITIONS':[
                (data['WILLR'] < 80),
                (data['WILLR'] > 20),
                ],
                 'TS4_CONDITIONS':[
                (data['MOM'] > 0 ),
                (data['MOM'] <= 0),
                ],
                 'TS5_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= 100),
                ],
                 'TS6_CONDITIONS': [
                (data['SLOWK'] > data['SLOWD']) & (data['SLOWD'] < 20),
                (data['SLOWK'] < data['SLOWD']) & (data['SLOWD'] > 80)],
                'TS7_CONDITIONS': [
                (data['MACD'] > 0 ),
                (data['MACD'] <= 0)],
                'TS8_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= -100)]}

    # create a list of the values we want to assign for each condition 1: buy, 0: sell
    values = [1, 0]

    # create a new column and use np.select to assign values to it using our lists as arguments
    for i in conditions:
        data[i] = np.select(conditions[i], values)
    strategies =list(conditions.keys())
    return data,strategies
data, strategies =generate_candidate_trading_signals(data.copy())

In [26]:
#split data
train = data[(data['Date'] <= date(2016,12,31))]
test = data[(data['Date'] > date(2016,12,31))]
train_data, strategies =generate_candidate_trading_signals(train.copy())
test_data, test_strategies =generate_candidate_trading_signals(test.copy())
num_islands=3
num_iter=50
pSize=50
m_iter=5
N=5
K=3
r_cross=0.8
r_mut= 0.01
r_inv=0.3
n=4
b=4
stop_loss=-0.15
take_profit=0.15
allocated_capital=1000
islandgga = IslandGGA(num_islands,num_iter,train_data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)

In [28]:
ring = []
master = []
multi = []
gga = []
for i in range(80):
    islandgga.evolve_island_ring()
    best  = islandgga.globalBest
    print(best)
    ring.append(best)
    islandgga.evolve_master_slave()
    best  = islandgga.globalBest
    print(best)
    master.append(best)
    islandgga.evolve_island_multikuti()
    best  = islandgga.globalBest
    print(best)
    multi.append(best)
    islandgga.evolve_gga()
    best  = islandgga.globalBest
    print(best)
    gga.append(best)
    print("ITERATION: {} DONE".format(i+1))
    print("========================")

SLTP: [1, 0, 1, 0, 1, 1, 0, 1]
GROUP: [['TS2_CONDITIONS', 'TS3_CONDITIONS', 'TS6_CONDITIONS'], ['TS7_CONDITIONS', 'TS8_CONDITIONS', 'TS4_CONDITIONS'], ['TS5_CONDITIONS', 'TS1_CONDITIONS']]
WEIGHT: [0, 0, 0, 0, 1, 1, 1]
FITNESS: 611.3069294501007
SLTP: [0, 1, 1, 1, 0, 1, 0, 0]
GROUP: [['TS2_CONDITIONS', 'TS3_CONDITIONS', 'TS5_CONDITIONS'], ['TS6_CONDITIONS', 'TS1_CONDITIONS', 'TS4_CONDITIONS'], ['TS7_CONDITIONS', 'TS8_CONDITIONS']]
WEIGHT: [0, 0, 0, 0, 1, 1, 1]
FITNESS: 399.9265100529263
SLTP: [1, 0, 0, 1, 0, 0, 1, 1]
GROUP: [['TS2_CONDITIONS', 'TS1_CONDITIONS', 'TS3_CONDITIONS'], ['TS6_CONDITIONS', 'TS7_CONDITIONS', 'TS4_CONDITIONS'], ['TS8_CONDITIONS', 'TS5_CONDITIONS']]
WEIGHT: [0, 0, 0, 0, 1, 1, 1]
FITNESS: 627.9586007244134
SLTP: [1, 1, 1, 0, 1, 1, 0, 0]
GROUP: [['TS8_CONDITIONS', 'TS4_CONDITIONS', 'TS5_CONDITIONS'], ['TS2_CONDITIONS', 'TS1_CONDITIONS', 'TS6_CONDITIONS'], ['TS3_CONDITIONS', 'TS7_CONDITIONS']]
WEIGHT: [1, 0, 1, 0, 1, 1, 1]
FITNESS: 312.99058882752274
ITERATION: 1 DO